In [1]:
#DataFrame
import pandas as pd
import numpy as np

#Time and date format handler
from datetime import datetime, timedelta

#Making API Requests
import requests

The goal is to obtain daily currency exchanges from a free API: https://exchangerate.host/#/

However, the API has a call limit  365 days. 

Thus, the aim of the script is to allow the users to fetch more years of data with one function and then concatenating the separate calls into one DataFrame.

The function works for multiples of 365 days, however this allows to have API calls for multiple years.

### Date delta check function

This fucntion can be used to check the length of days between 2 dates. 

However, the function get_exchange() works best for multiples of 365 days 

In [2]:
def get_delta(start,end):
    #Defining correct dateformat YYYY-MM-DD
    date_format = "%Y-%m-%d"
    #Convert to datetime format
    start = datetime.strptime(start, date_format)
    end = datetime.strptime(end, date_format)
    #Calculate the delta (days difference)
    delta = end-start
    print("The rounded delta between days divided by 365 is:", round(delta.days/365))
    print("The delta between the dates is:", delta.days)

In [3]:
#An example of the function
get_delta("2017-01-01", "2019-01-01")

The rounded delta between days divided by 365 is: 2
The delta between the dates is: 730


### Main function

In [4]:
def get_exchange(start,end,base="EUR"):
    '''
    get_exchange(start,end,base="EUR")
    Returns a DataFrame with Daily Exchange Rates for 137 currencies against default=EUR.
    Specify start, end in format YYYY-MM-DD.
    The function can span a time delta higher than the API limit of 365 days.
    '''
    
    #Defining correct dateformat YYYY-MM-DD
    date_format = "%Y-%m-%d"
    #Convert to datetime format
    start = datetime.strptime(start, date_format)
    end = datetime.strptime(end, date_format)
    #Obtain difference between start and end
    delta = end-start
    
    if delta.days > 365:
        #----First Call to the API---
        url = 'https://api.exchangerate.host/timeseries?start_date=' + str(start) + '&end_date='+ str(end)+'&base='+str(base)
        response = requests.get(url)
        data = response.json()
        df1 = pd.DataFrame(list(data.items())[6][1]).transpose()
        #Set multiple of the days in years excessing the threshold of 365
        k = round(delta.days/365)
        #Setting empty dataframe to collect data
        df_add = pd.DataFrame()
        
        for i in range(1,k):
            #---Start----
            #Adding multiples of 365 to get extra days
            start_new = start + timedelta(days=365)*i
            #Convert datetime back to string
            start_new = start_new.strftime(date_format)
            #---End----
            #Adding multiples of 365 to get extra days
            end_new = end + timedelta(days=365)*i
            #Convert datetime back to string
            end_new = end_new.strftime(date_format)
            
            #---API Additional Calls---
            #Add start date string back to url call
            url = 'https://api.exchangerate.host/timeseries?start_date=' + start_new+'&end_date='+ end_new+'&base='+str(base)
            response = requests.get(url)
            data = response.json()
            df_loop = pd.DataFrame(list(data.items())[6][1]).transpose()
            #append data to DataFrame
            df_add = df_add.append(df_loop)
            
        #Append first call data to successive calls
        dffinal = pd.concat([df1,df_add])
        return dffinal
    else:
        #Convert back to string datetime
        start_new = start.strftime(date_format)
        end_new = end.strftime(date_format)
        #----First Call to the API---
        url = 'https://api.exchangerate.host/timeseries?start_date=' + start_new + '&end_date='+end_new+'&base='+str(base)
        response = requests.get(url)
        data = response.json()
        df = pd.DataFrame(list(data.items())[6][1]).transpose()
        return df

First, the function is called for 2 dates which have a range higher than 365 days (the standard limit of the API).
It can be seen that the collected data ranges for the whole requeste time period

In [5]:
#Here we create an example DataFrame where we call the API throught the created function for the user 
#specified date ranges
ex_EUR = get_exchange("2017-01-01", "2019-01-01")

In [6]:
ex_EUR

,USD,JPY,BGN,CZK,DKK,GBP,HUF,PLN,RON,SEK,...,XCD,XOF,XPF,YER,ZMW,ANG,AWG,CDF,BYN,CUC
2017-01-01,1.0575,122.926,1.9484,27.0144,7.4314,0.8603,310.861,4.4113,4.5286,9.5840,...,2.8428,655.957,119.26,264.118,10.2368,NaN,NaN,NaN,NaN,NaN
2017-01-02,1.0527,122.930,1.9468,27.0181,7.4323,0.8514,308.968,4.4007,4.5347,9.5599,...,2.8299,655.957,119.26,262.911,10.4305,NaN,NaN,NaN,NaN,NaN
2017-01-03,1.0460,122.870,1.9482,27.0155,7.4320,0.8515,309.139,4.4092,4.5297,9.5363,...,2.8119,655.957,119.26,261.239,10.3504,NaN,NaN,NaN,NaN,NaN
2017-01-04,1.0420,122.570,1.9484,27.0155,7.4331,0.8508,309.319,4.3854,4.5155,9.5355,...,2.8011,655.957,119.26,260.238,10.2611,NaN,NaN,NaN,NaN,NaN
2017-01-05,1.0501,122.957,1.9482,27.0155,7.4344,0.8519,307.795,4.3687,4.5027,9.5495,...,2.8228,655.957,119.26,262.260,10.2877,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,1.1361,126.405,1.9557,25.9010,7.4636,0.8982,321.038,4.2810,4.6390,10.3273,...,NaN,655.960,119.26,NaN,13.5410,NaN,NaN,NaN,NaN,NaN
2018-12-28,1.1431,126.768,1.9479,25.8103,7.4658,0.9039,320.710,4.2938,4.6519,10.2999,...,NaN,655.960,119.26,NaN,13.5310,NaN,NaN,NaN,NaN,NaN
2018-12-29,1.1431,126.768,1.9479,25.8103,7.4658,0.9039,320.710,4.2938,4.6519,10.2999,...,NaN,655.960,119.26,NaN,13.5310,NaN,NaN,NaN,NaN,NaN
2018-12-30,1.1431,126.768,1.9479,25.8103,7.4658,0.9039,320.710,4.2938,4.6519,10.2999,...,NaN,655.960,119.26,NaN,13.5310,NaN,NaN,NaN,NaN,NaN


Here the same time period, but the base currency USD is specified.

In [131]:
ex_USD = get_exchange("2017-01-01", "2019-01-01", base = "USD")

In [132]:
ex_USD

,USD,JPY,BGN,CZK,DKK,GBP,HUF,PLN,RON,SEK,...,XCD,XOF,XPF,YER,ZMW,ANG,AWG,CDF,BYN,CUC
2017-01-01,1.0,116.242080,1.842459,25.545532,7.027329,0.813522,293.958392,4.171442,4.282364,9.062884,...,2.688227,620.290307,112.775414,249.756974,9.680189,NaN,NaN,NaN,NaN,NaN
2017-01-02,1.0,116.775910,1.849340,25.665527,7.060226,0.808777,293.500522,4.180393,4.307685,9.081315,...,2.688230,623.118647,113.289636,249.749216,9.908331,NaN,NaN,NaN,NaN,NaN
2017-01-03,1.0,117.466539,1.862524,25.827438,7.105163,0.814054,295.543977,4.215296,4.330497,9.116922,...,2.688241,627.109943,114.015296,249.750478,9.895220,NaN,NaN,NaN,NaN,NaN
2017-01-04,1.0,117.629559,1.869866,25.926583,7.133493,0.816507,296.851248,4.208637,4.333493,9.151152,...,2.688196,629.517274,114.452975,249.748560,9.847505,NaN,NaN,NaN,NaN,NaN
2017-01-05,1.0,117.090753,1.855252,25.726597,7.079707,0.811256,293.110180,4.160270,4.287877,9.093896,...,2.688125,624.661461,113.570136,249.747643,9.796876,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1.0,109.611747,1.755157,22.918566,6.727232,0.769120,298.061436,3.835600,4.303937,9.405729,...,NaN,590.899018,107.431763,NaN,13.067652,NaN,NaN,NaN,NaN,NaN
2019-12-28,1.0,109.611747,1.755157,22.918566,6.727232,0.769120,298.061436,3.835600,4.303937,9.405729,...,NaN,590.899018,107.431763,NaN,13.067652,NaN,NaN,NaN,NaN,NaN
2019-12-29,1.0,109.611747,1.755157,22.918566,6.727232,0.769120,298.061436,3.835600,4.303937,9.405729,...,NaN,590.899018,107.431763,NaN,13.067652,NaN,NaN,NaN,NaN,NaN
2019-12-30,1.0,109.458952,1.742354,22.737614,6.679306,0.763817,295.695761,3.809426,4.276337,9.325434,...,NaN,586.618673,106.653550,NaN,13.787695,NaN,NaN,NaN,NaN,NaN


Exporting to CSV file

In [133]:
ex_EUR.to_csv("daily_ex_rates.csv")